In [2]:
# !pip install tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import deque
import random
from qnetwork import DQNetwork
from dataset import Cifar10ImageDataset
from agent import DQNAgent

In [3]:
dataset = Cifar10ImageDataset(20)


Reward for each class.
	- Class 0 : 0.867110
	- Class 1 : 0.433555
	- Class 2 : 0.086711
	- Class 3 : 0.086711
	- Class 4 : 0.086711
	- Class 5 : 0.086711
	- Class 6 : 0.086711
	- Class 7 : 0.086711
	- Class 8 : 0.086711
	- Class 9 : 0.086711


In [4]:
network = DQNetwork(state_size=(32, 32, 3), action_size=10, gamma = 0.95, epsilon = 1.0, learning_rate = 0.001)

Num GPUs Available:  0



In [5]:
agent = DQNAgent(network, dataset, state_size=(32, 32, 3), action_size = 10, memory=deque(maxlen=100), epsilon=1.0)

In [6]:
agent.train(10)

Action is 1 and label is 6
Reward is -0.086711
Action is 3 and label is 9
Reward is -0.086711
Action is 5 and label is 7
Reward is -0.086711
Action is 0 and label is 4
Reward is -0.086711
Action is 3 and label is 8
Reward is -0.086711
Action is 8 and label is 3
Reward is -0.086711
Action is 7 and label is 7
Reward is 0.086711
Action is 0 and label is 8
Reward is -0.086711
Action is 3 and label is 4
Reward is -0.086711
Action is 6 and label is 1
Reward is -0.433555
Episode: 0, Reward: -0.433555
Action is 6 and label is 7
Reward is -0.086711
Action is 6 and label is 8
Reward is -0.086711
Action is 6 and label is 5
Reward is -0.086711
Action is 9 and label is 5
Reward is -0.086711
Action is 5 and label is 7
Reward is -0.086711
Action is 5 and label is 8
Reward is -0.086711
Action is 7 and label is 5
Reward is -0.086711
Action is 8 and label is 2
Reward is -0.086711
Action is 4 and label is 2
Reward is -0.086711
Action is 8 and label is 6
Reward is -0.086711
Action is 6 and label is 6
Rewa

: 

In [ ]:
agent.evaluate()  